### Loading Data into S3 through APIs

In [ ]:
# AWS Libraries/APIs
# import boto3
# import sagemaker
# import botocore
# low-level service client of the boto3 session
# sm = boto3.client(service_name='sagemaker', 
#                   config=config)

# sm_runtime = boto3.client('sagemaker-runtime',
#                           config=config)

# sess = sagemaker.Session(sagemaker_client=sm,
#                          sagemaker_runtime_client=sm_runtime)

# bucket = sess.default_bucket()
# role = sagemaker.get_execution_role()
# region = sess.boto_region_name

# s3 = boto3.Session().client(service_name='s3', 
#                             config=config)

# from sagemaker.s3 import S3Downloader, S3Uploader

# s3_uri = 's3://projects/lockdown-predictor/'
# data_uri = s3_uri + 'data/' 
# S3Downloader().download(data_uri, 'data/')

# S3Uploader().upload(data_uri, 'data/')

### Twitter API

In [ ]:
import yaml, requests
try:
    creds = yaml.safe_load(open('credentials.yml'))
    twitter_creds = creds['twitter']

    TWITTER_API_KEY = twitter_creds['API_KEY']
    TWITTER_API_SECRET_KEY = twitter_creds['API_SECRET_KEY']
    TWITTER_BEARER_TOKEN = twitter_creds['BEARER_TOKEN']
except Exception:
    raise Exception('''If you want twitter data you will need to setup your keys in a credentials.yml file
    twitter:
        API_KEY: XXX
        API_SECRET_KEY: XXX
        BEARER_TOKEN: XXX''')
    
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = TWITTER_BEARER_TOKEN

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {
    'query': '(from:twitterdev -is:retweet) OR #twitterdev',
    'tweet.fields': 'context_annotations,created_at,public_metrics',
    'expansions': ''
}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
    
json_response = connect_to_endpoint(search_url, query_params)